## 1. Import libraries

In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set_theme() # set the default Seaborn style for graphics

## 2. Data cleaning

### Data cleaning on the raw dataset (spotify_songs.csv) and extract to save it as cleaned_dataset.csv

In [ ]:
# Import dataset and check basic information
full_dataset = pd.read_csv('datasets/spotify_songs.csv')
print(full_dataset.shape)
# print(full_dataset.dtypes)
# full_dataset.head()
# full_dataset.describe()
full_dataset.info()

From the printed `.info()` above, we can see that some of the song records in the `full_dataset` contain `NA` values in certain variable columns. Hence, we drop those song records from `full_dataset`.

In [ ]:
# Apply the dropna() function to remove records with missing values
# Then check the information of the cleaned dataset
full_dataset.dropna(inplace=True)
full_dataset.info()

Our goal is to predict the popularity of the songs using different variables as predictors.
In order to achieve this:
1. Analyse the 'track_popularity' variable
2. Categorize the track_popularity score into 6 different categories.

In [ ]:
# Extract the track_popularity column and check its distribution
popularity = pd.DataFrame(full_dataset["track_popularity"])
print(popularity.describe())
print(popularity.value_counts())

f, axes = plt.subplots(3, 1, figsize=(24, 12), sharex=True)
sb.boxplot(data = popularity, orient = "h", ax=axes[0])
sb.histplot(data = popularity, kde = True, ax=axes[1])
sb.violinplot(data = popularity, orient = "h", ax=axes[2])

From the plots and `.value_counts()` above, we can see that there are unexpectedly high number of song records with `track_popularity <= 1`. In order to ensure the datas are more normally distributed, we choose to remove those song records from our `full_dataset`.

In [ ]:
# Remove records whose value of track_popularity is lower than or equal to 1
full_dataset = full_dataset[full_dataset['track_popularity'] > 1]
popularity = full_dataset["track_popularity"]

# Check the skewness and distribution of the track_popularity column
from scipy.stats import skew

print("Skewness of popularity:", skew(popularity))
print(popularity.value_counts())
f, axes = plt.subplots(3, 1, figsize=(24, 12), sharex=True)
sb.boxplot(data = popularity, orient = "h", ax=axes[0])
sb.histplot(data = popularity, kde = True, ax=axes[1])
sb.violinplot(data = popularity, orient = "h", ax=axes[2])

From the calculated skewness and plots above, we can clearly see that **`track_popularity` is similar to a normal distribution** if we discard the clustering phenomenon at the lower spectrum.

Thus, we can then divide `track_popularity` into **6 levels** -- `very_low`, `low`, `somewhat_low`, `somewhat_high`, `high`, and `very_high` -- using mean and standard deviation as the parameters to gauge.

In [ ]:
# Calculate the mean and standard deviation of track_popularity
mean = popularity.mean()
std = popularity.std()

# Define the level divisions
very_low = mean - 2 * std
low = mean - std
medium = mean
high = mean + std
very_high = mean + 2 * std

# Create a new column "popularity_level" based on the level divisions
full_dataset["popularity_level"] = pd.cut(full_dataset["track_popularity"], bins=[0, very_low, low, medium, high, very_high, float('inf')], labels=["very_low", "low", "somewhat_low", "somewhat_high", "high", "very_high"])

# Check the distribution of popularity levels
popularity_level = pd.DataFrame(full_dataset["popularity_level"].value_counts(sort=False), columns=["count"])
popularity_level["density"] = popularity_level["count"] / len(full_dataset)
popularity_level

We then save the cleaned dataset to a new CSV file named clean_dataset.csv

In [ ]:
#Save the extracted and cleaned dataset into a separate file named clean_dataset.csv
full_dataset.to_csv('datasets/cleaned_dataset.csv', index=False)

## 3. Data randomization

The current sample size is over 30k, and we choose to reduce it to 5k by random sampling method and save the randomized dataset to a new CSV file named random_sampled_dataset.csv

In [ ]:
sampled_dataset = full_dataset.sample(n=5000, random_state=29) # Use a random seed of 29 for reproducibility
sampled_dataset.to_csv('datasets/random_sampled_dataset.csv', index=False)
print(sampled_dataset.shape)
print(sampled_dataset["popularity_level"].value_counts(sort=False) / len(sampled_dataset))

## 4. EDA and Visualisation analysis on random_sampled.csv

### Out of all variables in the random_sampled.csv file, we have found several numeric as well as categorical variables for analysis

### EDA and Visualisation analysis on numeric variables

We first examine the numeric variables and their relationships with track_popularity

In [ ]:
numeric_columns = ["track_popularity", "danceability", "energy", "speechiness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms"]

print(sampled_dataset[numeric_columns].corr())
sb.heatmap(sampled_dataset[numeric_columns].corr(), vmin = -1, vmax = 1, annot = True, fmt=".2f")

f, axes = plt.subplots(3, 1, figsize=(24, 24), sharex=True)
sb.boxplot(data = sampled_dataset[numeric_columns].apply(min_max_normalize), orient = "h", ax=axes[0])
sb.histplot(data = sampled_dataset[numeric_columns].apply(min_max_normalize), kde = True, ax=axes[1])
sb.violinplot(data = sampled_dataset[numeric_columns].apply(min_max_normalize), orient = "h", ax=axes[2])

From the heatmap, we find 4 variables with the highest correlations with variable track_popularity (Highest correlation is determined by the highest absolute values of different variables with track_popularity)
The 4 variables are: instrumentalness (correlation with track_popularity = -0.16), energy (correlation with track_popularity = -0.12), duration_ms (correlation with track_popularity = -0.10), danceability (correlation with track_popularity = 0.07)

In [ ]:
def min_max_normalize(x):
    return (x - x.min()) / (x.max() - x.min()) * 100

numeric5_columns = ["track_popularity", "instrumentalness", "energy", "duration_ms","danceability"]
print(sampled_dataset[numeric5_columns].corr())
sb.heatmap(sampled_dataset[numeric5_columns].corr(), vmin = -1, vmax = 1, annot = True, fmt=".2f")

f, axes = plt.subplots(3, 1, figsize=(24, 36), sharex=True)
sb.boxplot(data = sampled_dataset[numeric5_columns].apply(min_max_normalize), orient = "h", ax=axes[0])
sb.histplot(data = sampled_dataset[numeric5_columns].apply(min_max_normalize), kde = True, ax=axes[1])
sb.violinplot(data = sampled_dataset[numeric5_columns].apply(min_max_normalize), orient = "h", ax=axes[2])

From various plots shown above, we can easily see that the variable instrumentalness is not a good variable to be used for eda analysis as the graph is severely skewed.
To further prove our point, we use .describe() functions to see the distribution of the variable

In [ ]:
print(sampled_dataset[["instrumentalness"]].describe())
print(sampled_dataset[["instrumentalness"]].value_counts())

The result above shows that there is a total of 1914 counts out of 5000 that have the instrumentalness value of 0. As there is a serious clustering at the low end, we decide to not use this variable for EDA analysis with track_popularity.

In [ ]:
def min_max_normalize(x):
    return (x - x.min()) / (x.max() - x.min()) * 100

numeric4_columns = [ "track_popularity", "energy", "duration_ms", "danceability"]
print(sampled_dataset[numeric4_columns].corr())
sb.heatmap(sampled_dataset[numeric4_columns].corr(), vmin = -1, vmax = 1, annot = True, fmt=".2f")

f, axes = plt.subplots(3, 1, figsize=(24, 36), sharex=True)
sb.boxplot(data = sampled_dataset[numeric4_columns].apply(min_max_normalize), orient = "h", ax=axes[0])
sb.histplot(data = sampled_dataset[numeric4_columns].apply(min_max_normalize), kde = True, ax=axes[1])
sb.violinplot(data = sampled_dataset[numeric4_columns].apply(min_max_normalize), orient = "h", ax=axes[2])

### EDA and Visualisation analysis on categorical variables

We then examine the categorical variables and their relationships with popularity_level.

In [ ]:
categorical_columns = ["playlist_genre", "playlist_subgenre", "mode", "key"]
sampled_dataset[categorical_columns] = sampled_dataset[categorical_columns].astype('category')
sampled_dataset[categorical_columns].describe()

In [ ]:
# playlist_genre vs popularity_level
f = plt.figure(figsize=(16, 8))
sb.heatmap(sampled_dataset.groupby(['popularity_level', 'playlist_genre']).size().unstack(),
            linewidths=1, annot = True, fmt = 'g', annot_kws={"size": 18}, cmap="BuGn")

In [ ]:
# playlist_subgenre vs popularity_level
f = plt.figure(figsize=(24, 8))
sb.heatmap(sampled_dataset.groupby(['popularity_level', 'playlist_subgenre']).size().unstack(),
            linewidths=1, annot = True, fmt = 'g', annot_kws={"size": 18}, cmap="BuGn")

In [ ]:
# mode vs popularity_level
f = plt.figure(figsize=(4, 8))
sb.heatmap(sampled_dataset.groupby(['popularity_level', 'mode']).size().unstack(),
            linewidths=1, annot = True, fmt = 'g', annot_kws={"size": 18}, cmap="BuGn")

In [ ]:
# key vs popularity_level
f = plt.figure(figsize=(24, 8))
sb.heatmap(sampled_dataset.groupby(['popularity_level', 'key']).size().unstack(),
            linewidths=1, annot = True, fmt = 'g', annot_kws={"size": 18}, cmap="BuGn")

From the diagrams above, we can see that out of all 4 categorical values listed above, 2 of them (mode and playlist_subgenre) display some trends that can be used as predictors. Hence, we will use these two categorical variables and fit them into the models subsequently.

## 5. Creating Model 1 for prediction of popularity_level